# 0. 패키지 불러오기

In [1]:
!pip install pandas
!pip install numpy
!pip install catboost
!pip install lightgbm
!pip install category_encoders
!pip install optuna
!pip install seaborn
!pip install matplot
!pip install imblearn

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import math
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import * # ExtraTreesRegressor, RandomForestClassifier
from sklearn.model_selection import * # KFold, cross_val_score, train_test_split
from sklearn.metrics import *  # make_scorer : MSE 대신 사용자가 정의한 손실함수를 사용하고 싶을 때
from sklearn.preprocessing import * # LabelEncoder, StandardScaler
from sklearn.svm import * # SVC, SVR
from sklearn.inspection import *
from sklearn.linear_model import * # LogisticRegression, RANSACRegressor, Ridge, Lasso, ElasticNet
from sklearn.decomposition import * # PCA

from category_encoders.ordinal import OrdinalEncoder
from catboost import *
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
import random
from imblearn.over_sampling import SMOTE, ADASYN

from lightgbm import *
from datetime import datetime
import calendar
from collections import *

import warnings
warnings.filterwarnings(action = 'ignore') # 경고메세지를 무시하거나 숨긴다.

In [3]:
!ls '/home/studio-lab-user/MYDATA'

'Construction Machine Oil'  'Getting Started.ipynb'   images


In [4]:
!ls '/home/studio-lab-user/MYDATA/Construction Machine Oil/open/'

data_info.xlsx	sample_submission.csv  test.csv  train.csv


In [5]:
path = '/home/studio-lab-user/MYDATA/Construction Machine Oil/open/'

Rdata_train = pd.read_csv(path + 'train.csv')
Rdata_test = pd.read_csv(path + 'test.csv')

print(Rdata_train.shape) # (14095, 54) -> 생각보다 데이터 수가 적다.
print(Rdata_train.size)
print(Rdata_test.shape) # (6041, 19)
print(Rdata_test.size)
Rdata_train.head()

(14095, 54)
761130
(6041, 19)
114779


,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,...,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,TRAIN_00000,COMPONENT3,1486,2011,7,200,0,3,93,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,154.0,75,0
1,TRAIN_00001,COMPONENT2,1350,2021,51,375,0,2,19,0,...,2.0,4.0,6.0,216.0,1454.0,0,NaN,44.0,652,0
2,TRAIN_00002,COMPONENT2,2415,2015,2,200,0,110,1,1,...,0.0,3.0,39.0,11261.0,41081.0,0,NaN,72.6,412,1
3,TRAIN_00003,COMPONENT3,7389,2010,2,200,0,8,3,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.3,7,0
4,TRAIN_00004,COMPONENT3,3954,2015,4,200,0,1,157,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.1,128,0


In [6]:
train1 = Rdata_train.copy()
test1 = Rdata_test.copy()

In [7]:
# print(train1.loc[:, train1.notnull().sum(axis = 0) == 14095].columns) # 35개
# notnull_columns = train1.loc[:, train1.notnull().sum(axis = 0) == 14095].columns

In [8]:
train2 = train1.loc[:, ['COMPONENT_ARBITRARY', 'YEAR', 'ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 
                    'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN', 'AL', 'BA', 'Y_LABEL']]
test2 = test1.drop(['ID'], axis = 1)

print(train2.shape) # 변수 21개
print(test2.shape) # 변수 18개
print(train2.columns)

(14095, 21)
(6041, 18)
Index(['COMPONENT_ARBITRARY', 'YEAR', 'ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO',
       'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40',
       'ZN', 'AL', 'BA', 'Y_LABEL'],
      dtype='object')


In [9]:
le1 = LabelEncoder()
le2 = LabelEncoder()

train2['COMPONENT_ARBITRARY_category'] = le1.fit_transform(train2['COMPONENT_ARBITRARY'])
train2['YEAR_category'] = le2.fit_transform(train2['YEAR'])

test2['COMPONENT_ARBITRARY_category'] = le1.transform(test2['COMPONENT_ARBITRARY'])
test2['YEAR_category'] = le2.transform(test2['YEAR'])

train3 = train2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)
test3 = test2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)

In [10]:
categorical_features = ['COMPONENT_ARBITRARY_category', 'YEAR_category']
numeric_features_train = ['ANONYMOUS_1', 'SAMPLE_TRANSFER_DAY',
                           'ANONYMOUS_2', 'AG', 'AL', 'B', 'BA', 'BE', 'CA', 'CO', 'CR', 'CU',
                           'FE', 'H2O', 'LI', 'MG', 'MN', 'MO', 'NA', 'NI', 'P', 'PB', 'PQINDEX',
                           'S', 'SB', 'SI', 'SN', 'TI', 'V', 'V40', 'ZN']

numeric_features_test = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 
                    'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']

print(train3.shape)
print(test3.shape)
print(len(categorical_features))
print(len(numeric_features_train))
print(len(numeric_features_test))

(14095, 21)
(6041, 18)
2
31
16


In [11]:
X_train = train3.drop(['Y_LABEL'], axis = 1)
y_train = train3['Y_LABEL']
X_test = test3

In [12]:
X_train

,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN,AL,BA,COMPONENT_ARBITRARY_category,YEAR_category
0,1486,200,0,0,13,78,888,0.0,16,1,6,8504,5,0,154.0,75,3,0,2,4
1,1350,375,0,0,0,31,2,0.0,0,0,0,19,0,0,44.0,652,2,0,1,14
2,2415,200,0,0,1,2,4,0.0,0,0,0,17,0,0,72.6,412,110,1,1,8
3,7389,200,0,0,0,1,37,0.0,1,0,0,44,0,0,133.3,7,8,0,2,3
4,3954,200,0,0,0,0,71,0.0,0,0,0,217,0,0,133.1,128,1,0,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,1616,200,0,0,0,3,23,0.0,0,0,0,35,0,0,135.4,16,2,1,2,7
14091,2784,200,0,0,0,2,12,0.0,0,224,0,9,0,0,117.5,1408,3,0,0,6
14092,1788,550,0,0,4,7,415,0.0,7,10,1,645,0,0,54.0,1301,6,1,2,1
14093,2498,550,0,0,0,170,19,0.0,0,1,0,11,0,0,44.3,652,2,0,1,2


In [13]:
ads = ADASYN(random_state = 42) # seed
X_train, y_train = ads.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_train))
print(X_train.shape, y_train.shape)

Resampled dataset shape Counter({1: 12934, 0: 12892})
(25826, 20) (25826,)


In [14]:
X_train = X_train.loc[:, ['COMPONENT_ARBITRARY_category', 'ANONYMOUS_1', 'YEAR_category', 'ANONYMOUS_2', 'AG',
                                        'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
                                        'V40', 'ZN', 'AL', 'BA']]
print(X_train.shape)

(25826, 20)


In [15]:
X_partrain, X_val, y_partrain, y_val = train_test_split(X_train, 
                                                        y_train, 
                                                        test_size = 0.3, 
                                                        random_state = 39, 
                                                        stratify = y_train)
print(X_partrain.shape)
print(X_val.shape)
print(y_partrain.shape)
print(y_val.shape)

(18078, 20)
(7748, 20)
(18078,)
(7748,)


In [91]:
def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        "n_estimators" : trial.suggest_int("n_estimators", 100, 1000),
        "max_depth" : trial.suggest_int("max_depth", 3, 10)
  }
    
    model = CatBoostClassifier(**params_cat)
    model.fit(X_partrain, y_partrain, eval_set = [(X_val, y_val)],
              early_stopping_rounds = 100, verbose = False, cat_features = categorical_features)

    cat_pred = model.predict(X_val)
    AUC = roc_auc_score(y_val, cat_pred)
    
    return AUC

In [92]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "cat_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 100)

[I 2022-11-25 11:55:31,304] A new study created in memory with name: cat_parameter_opt
[I 2022-11-25 11:56:03,412] Trial 0 finished with value: 0.9559903357178647 and parameters: {'learning_rate': 0.06205093309656499, 'n_estimators': 818, 'max_depth': 9}. Best is trial 0 with value: 0.9559903357178647.
[I 2022-11-25 11:56:14,760] Trial 1 finished with value: 0.8654928357445174 and parameters: {'learning_rate': 0.008771416592003579, 'n_estimators': 642, 'max_depth': 7}. Best is trial 0 with value: 0.9559903357178647.
[I 2022-11-25 11:56:27,118] Trial 2 finished with value: 0.9369085757843901 and parameters: {'learning_rate': 0.04234350615938281, 'n_estimators': 524, 'max_depth': 8}. Best is trial 0 with value: 0.9559903357178647.
[I 2022-11-25 11:56:50,350] Trial 3 finished with value: 0.9548185481721555 and parameters: {'learning_rate': 0.35505674331734294, 'n_estimators': 834, 'max_depth': 10}. Best is trial 0 with value: 0.9559903357178647.
[I 2022-11-25 11:57:07,557] Trial 4 finishe

In [93]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.9621791010565145
Best trial : {'learning_rate': 0.14838330374220435, 'n_estimators': 968, 'max_depth': 10}


In [94]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [95]:
# 파라미터들과의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [96]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [16]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

cat_val = np.zeros((X_train.shape[0], 2))
cat_partrain = np.zeros((X_partrain.shape[0], 2))

print(cat_val.shape)
print(cat_partrain.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_cat = CatBoostClassifier(
        random_state = 39,
        learning_rate = 0.14838330374220435, 
        n_estimators = 968, 
        max_depth = 8)

    optuna_cat.fit(X_train.loc[i_trn, :], y_train[i_trn], verbose = False)

    cat_val[i_val, :] = optuna_cat.predict_proba(X_train.loc[i_val, :])
    cat_partrain += optuna_cat.predict_proba(X_partrain) / n_fold

(25826, 2)
(18078, 2)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [17]:
print(cat_val.shape)
print(cat_partrain.shape)

X_train['model1_prob'] = cat_val[:, 1]
print(X_train.shape)

(25826, 2)
(18078, 2)
(25826, 21)


In [18]:
X_train2 = X_train.drop(['AL', 'BA', 'model1_prob'], axis = 1)
y_train2 = X_train['model1_prob']
print(X_train2.shape)
print(y_train2.shape)

(25826, 18)
(25826,)


In [19]:
X_partrain, X_val, y_partrain, y_val = train_test_split(X_train2, y_train2, test_size = 0.3, random_state = 39)
print(X_partrain.shape)
print(X_val.shape)
print(y_partrain.shape)
print(y_val.shape)

(18078, 18)
(7748, 18)
(18078,)
(7748,)


In [101]:
def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        "n_estimators" : trial.suggest_int("n_estimators", 100, 1000),
        "max_depth" : trial.suggest_int("max_depth", 3, 10)
  }
    
    model = CatBoostRegressor(**params_cat)
    model.fit(X_partrain, y_partrain, eval_set = [(X_val, y_val)],
              early_stopping_rounds = 100, verbose = False,  cat_features = categorical_features)

    cat_pred = model.predict(X_val)
    MAE = mean_absolute_error(y_val, cat_pred)
    
    return MAE

In [ ]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "cat_parameter_opt",
    direction = "minimize",
    sampler = sampler)
study.optimize(objective, n_trials = 100)

[I 2022-11-25 12:37:44,841] A new study created in memory with name: cat_parameter_opt
[I 2022-11-25 12:38:03,836] Trial 0 finished with value: 0.2221259236189402 and parameters: {'learning_rate': 0.06205093309656499, 'n_estimators': 818, 'max_depth': 9}. Best is trial 0 with value: 0.2221259236189402.
[I 2022-11-25 12:38:09,903] Trial 1 finished with value: 0.3523152542685131 and parameters: {'learning_rate': 0.008771416592003579, 'n_estimators': 642, 'max_depth': 7}. Best is trial 0 with value: 0.2221259236189402.
[I 2022-11-25 12:38:16,951] Trial 2 finished with value: 0.27176471918164236 and parameters: {'learning_rate': 0.04234350615938281, 'n_estimators': 524, 'max_depth': 8}. Best is trial 0 with value: 0.2221259236189402.
[I 2022-11-25 12:38:40,881] Trial 3 finished with value: 0.19302179890951432 and parameters: {'learning_rate': 0.35505674331734294, 'n_estimators': 834, 'max_depth': 10}. Best is trial 3 with value: 0.19302179890951432.
[I 2022-11-25 12:38:51,924] Trial 4 fini

In [59]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.19449510761199787
Best trial : {'learning_rate': 0.22259744286979857, 'n_estimators': 702, 'max_depth': 10}


In [60]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [61]:
# 파라미터들과의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [62]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [20]:
n_fold = 5
cv = KFold(n_splits = n_fold, shuffle = True, random_state = 39)

cat_val = np.zeros((X_train2.shape[0]))
cat_test = np.zeros((X_test.shape[0]))

print(cat_val.shape)
print(cat_test.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train2, y_train2), 1):
    print(f'training model for CV #{i}')
    optuna_cat = CatBoostRegressor(
        random_state = 39,
        learning_rate = 0.2092401876135517, 
        n_estimators = 903, 
        max_depth = 10)

    optuna_cat.fit(X_train2.loc[i_trn, :], y_train2[i_trn], verbose = False)

    cat_val[i_val] = optuna_cat.predict(X_train2.loc[i_val, :])
    cat_test += optuna_cat.predict(X_test) / n_fold

(25826,)
(6041,)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [21]:
scores = []
for threshold in range(50) :
    threshold = threshold / 50
    pred = cat_val
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(y_train, pred)
    scores.append(score)

temp1 = pd.DataFrame(np.linspace(0, 0.98, 50), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
scores = pd.concat([temp1, temp2], axis = 1)
scores


,threshold,score
0,0.00,0.720824
1,0.02,0.728555
2,0.04,0.736536
3,0.06,0.746098
4,0.08,0.755769
5,0.10,0.764596
6,0.12,0.773775
7,0.14,0.782498
8,0.16,0.791106
9,0.18,0.801819


In [22]:
answer = np.zeros(cat_test.shape[0])

for i in range(cat_test.shape[0]) :
  if cat_test[i] >= 0.56 :
    answer[i] = 1

Counter(answer)

Counter({0.0: 5488, 1.0: 553})

In [23]:
# 제출

submission_preds = answer
submission = pd.read_csv('/home/studio-lab-user/MYDATA/Construction Machine Oil/open/sample_submission.csv')
submission['Y_LABEL'] = submission_preds
submission.to_csv('/home/studio-lab-user/MYDATA/Construction Machine Oil/결과/submission_catboost_ADASYN1.csv', index = False)